In [1]:
import logging
import subprocess
import json
import argparse
from typing import Dict, List
import shutil
import ray
from ray.util import ActorPool
import os
import sys
import gc

from rcnn_iwp_inference import load_model, get_metadata, run_inference


/projects/bbou/mohanar2/envs/ray_sandbox/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-23 08:00:38,467	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
@ray.remote(num_cpus=1, memory=4 * 1024 * 1024 * 1024)
class ImageInferenceActor:
    def __init__(self, model_ref, metadata_ref):
        self.model = model_ref
        self.metadata = metadata_ref

    def process_image(self, image_path, threshold, output_folder, worker_id):
        try:
            print(f"Worker {worker_id} running inference on {image_path}")
            
            image_file = os.path.basename(image_path)
            
            # Create unique temporary output file using worker_id and image name
            temp_output_file = f"output_{worker_id}_{image_file}.jpg"
            
            # Run inference with custom output path
            coco_mask_metadata = run_inference(
                self.model, 
                image_path, 
                self.metadata, 
                threshold=threshold, 
                output_file=temp_output_file
            )
            
            # Create output file path
            output_file = os.path.join(output_folder, f"{image_file.split('.')[0]}_masked.jpg")
            
            # Copy the output image to the output folder
            shutil.copy(temp_output_file, output_file)
            os.remove(temp_output_file)
            
            # Force garbage collection to free memory
            gc.collect()
            
            return {"file": image_file, "metadata": coco_mask_metadata, "output_file": output_file}
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            # Force garbage collection even on error
            gc.collect()
            return {"file": os.path.basename(image_path), "error": str(e)}

In [4]:
# Load the model

model_file = "./model_weights.pth"
model = load_model("configs/mask_rcnn_vitdet.py", "configs/datasets/iwp.py", model_file)
metadata = get_metadata("iwp_test", "data/iwp/iwp_test.json")


/projects/bbou/mohanar2/envs/ray_sandbox/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/projects/bbou/mohanar2/envs/ray_sandbox/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [5]:
# Initialize Ray
HEAD_NODE = os.environ["HEAD_NODE"]
RAY_PORT = os.environ["RAY_PORT"]

ray.init(address=f"{os.environ['HEAD_NODE']}:{os.environ['RAY_PORT']}",_node_ip_address=os.environ['HEAD_NODE'],
         ignore_reinit_error=True)


print(ray.available_resources())



2025-06-23 08:02:24,514	INFO worker.py:1723 -- Connecting to existing Ray cluster at address: cn062.delta.ncsa.illinois.edu:34567...
2025-06-23 08:02:24,547	INFO worker.py:1908 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


{'node:cn062.delta.ncsa.illinois.edu': 1.0, 'object_store_memory': 75745941504.0, 'node:__internal_head__': 1.0, 'memory': 176740530176.0, 'CPU': 16.0}


In [6]:
#  Add to store
model_ref = ray.put(model)
metadata_ref = ray.put(metadata)


In [7]:
output_folder_name = f"output_images_masked"
output_folder = os.path.join(os.getcwd(), output_folder_name)
os.makedirs(output_folder, exist_ok=True)

In [8]:
dataset_folder = 'test_images'
dataset_folder = os.path.join(os.getcwd(), dataset_folder)
image_files = [os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder)]

print(image_files)

['/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_280_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_289_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_291_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_292_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_294_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_301_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_303_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_308_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_310_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_320_Polygon_1.jpg', '/u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_337_Polygon_2.jpg', '/u/mohanar2/pdg-ext

In [9]:
NUM_ACTORS = 4
actors = [ImageInferenceActor.remote(model_ref, metadata_ref) for _ in range(NUM_ACTORS)]
actors_pool = ActorPool(actors)

In [10]:
# Create task inputs with unique IDs
tasks = [
    (image_path, 0.6, output_folder, i) 
    for i, image_path in enumerate(image_files)
]

# Process all images using the actor pool
results = list(actors_pool.map(lambda actor, task: actor.process_image.remote(*task), tasks))

(ImageInferenceActor pid=2394822) /projects/bbou/mohanar2/envs/ray_sandbox/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
(ImageInferenceActor pid=2394822)   check_for_updates()


(ImageInferenceActor pid=2394822) Worker 0 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_280_Polygon_1.jpg


(ImageInferenceActor pid=2394822) /projects/bbou/mohanar2/envs/ray_sandbox/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
(ImageInferenceActor pid=2394822)   return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
(ImageInferenceActor pid=2394832) /projects/bbou/mohanar2/envs/ray_sandbox/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1. [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for mo

(ImageInferenceActor pid=2394822) Worker 4 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_294_Polygon_1.jpg [repeated 4x across cluster]
(ImageInferenceActor pid=2394822) Worker 8 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_310_Polygon_1.jpg [repeated 4x across cluster]
(ImageInferenceActor pid=2394822) Worker 12 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_346_Polygon_2.jpg [repeated 4x across cluster]
(ImageInferenceActor pid=2394822) Worker 16 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_362_Polygon_2.jpg [repeated 4x across cluster]
(ImageInferenceActor pid=2394822) Worker 20 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_385_Polygon_2.jpg [repeated 4x across cluster]
(ImageInferenceActor pid=2394822) Worker 24 running inference on /u/mohanar2/pdg-extractors/rcnn_iwp_i

In [11]:
print(results)

[{'file': 'FID_280_Polygon_1.jpg', 'metadata': [{'id': 1, 'category_id': 1, 'bbox': [440.8972473144531, 379.2193603515625, 109.91891479492188, 114.6072998046875], 'area': 12597.509765625, 'score': 0.9871485829353333}, {'id': 2, 'category_id': 1, 'bbox': [208.4915771484375, 360.42791748046875, 129.9141845703125, 118.50762939453125], 'area': 15395.822265625, 'score': 0.9841552972793579}, {'id': 3, 'category_id': 1, 'bbox': [385.6973571777344, 246.5799560546875, 115.89422607421875, 123.7130126953125], 'area': 14337.6240234375, 'score': 0.9605230689048767}, {'id': 4, 'category_id': 1, 'bbox': [82.33405303955078, 215.51979064941406, 141.70166015625, 106.92243957519531], 'area': 15151.0869140625, 'score': 0.9577971696853638}, {'id': 5, 'category_id': 1, 'bbox': [312.78515625, 38.24861145019531, 93.7591552734375, 118.40438842773438], 'area': 11101.4951171875, 'score': 0.9571066498756409}, {'id': 6, 'category_id': 1, 'bbox': [476.4351806640625, 275.3699035644531, 138.4271240234375, 121.1635742